In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
import string
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
## store the embeddings and one hot encodings using np.save()

In [ ]:
X_train,y_train,X_test,y_test = np.load('embed_train.npy'),np.load('labels_train.npy'),np.load('embed_test.npy'),np.load('labels_test.npy')

In [ ]:
alphabets = list(string.ascii_uppercase)

In [ ]:
# store both the multi label binarizer models

In [ ]:
with open('mlb.pkl','rb') as file:
    mlb = pickle.load(file)
with open('mlb_letter.pkl','rb') as file:
    mlb_letter = pickle.load(file)

In [ ]:
#y_train_letter
y_labels_train = mlb.inverse_transform(y_train)
y_labels_test = mlb.inverse_transform(y_test)

In [ ]:
y_starter_train = [tuple(s[0] for s in tup) for tup in y_labels_train]
y_starter_test = [tuple(s[0] for s in tup) for tup in y_labels_test]

In [ ]:
y_train_letter = mlb_letter.transform(y_starter_train)
y_test_letter = mlb_letter.transform(y_starter_test)

In [ ]:
gc.collect()

In [ ]:
###  The first stage begins ###
from skmultilearn.adapt import MLARAM

classifier = MLARAM(threshold=0.05, vigilance=0.95)
classifier.fit(X_train, y_train_letter)

In [ ]:
prediction = classifier.predict(X_test[:100])

In [ ]:
pred_1 = prediction

In [ ]:
from sklearn.metrics import fbeta_score
print(f"Score is {fbeta_score(prediction,y_test_letter[:100],beta = 2,average = 'macro')}")

In [ ]:
### The first stage ends ###

In [ ]:
### The second stage begins ###

In [ ]:
def filter_data_points(embeddings,labels, labels_to_check):
    #choose rows who have atleast one entry in 'labels_to_check' and choose labels data only from 'labels_to_check' columns
    interested_rows = np.any(labels[:, labels_to_check], axis=1)
    interested_labels = labels[interested_rows]
    filtered_labels = np.zeros((np.sum(interested_rows),labels.shape[1]))
    filtered_labels[:,labels_to_check] += interested_labels[:,labels_to_check] 
    filtered_embeddings = embeddings[interested_rows]
    return filtered_embeddings, filtered_labels

In [ ]:
#Training a model for each alphabet
mlabels = mlb.classes_
models = {}
for letter in tqdm(alphabets):
    # X should be embeddings, y should be multilabels
    indices  = [ind for ind,s in enumerate(mlabels) if s.startswith(letter)]
    filtered_data,filtered_labels = filter_data_points(X_train,y_train,indices)
    if len(indices) == 0:
        models[letter] = []
        continue
    elif len(indices) == 1:
        mask = np.zeros((1,len(mlabels)))
        mask[0,indices[0]] = 1
        models[letter] = mask
    else:
        model = MultiOutputClassifier(RandomForestClassifier(n_estimators = 5,verbose = 1))
        model.fit(filtered_data,filtered_labels)
        models[letter] = model

In [ ]:
### second stage ends ###

In [ ]:
### The final prediction ###

In [ ]:
def final_pred(X_test,y_actual,pred_1,models):
    pred_2 = np.zeros((X_test.shape[0],len(mlabels)))
    ind = 0
    for letter in tqdm(alphabets):
        if letter == 'P' or letter == 'U':
            continue
        reqd_data = np.any(pred_1[:,ind].reshape(-1,1),axis = 1)  #all rows where the pred_1 is 1 for the given letter
        reqd_model = models[letter]
        ind +=1
        if reqd_model == [] or np.sum(reqd_data) == 0:
            continue
        elif isinstance(reqd_model,type([])):
            pred_2[reqd_data,:] +=reqd_model
        else:
            pred_2[reqd_data,:] += reqd_model.predict(X_test[reqd_data,:])
    return pred_2

In [ ]:
pred_2 = final_pred(X_test[:100],y_test[:100],pred_1,models)

In [ ]:
print(f"Score is {fbeta_score(pred_2,y_test[:100],beta = 2,average = 'macro')}")

In [ ]:
### A few comparisons ###

In [ ]:
mlb.inverse_transform(pred_2[:5])

In [ ]:
y_labels_test[:5]